<a href="https://colab.research.google.com/github/Kwonmoment/LLaMA-3.1-fine-tuning-with-financial-QA-dataset/blob/main/%EC%9E%AC%EC%A0%95%EB%8D%B0%EC%9D%B4%ED%84%B0_%EC%A7%88%EC%9D%98%EC%9D%91%EB%8B%B5_llm_%EB%B9%84%EA%B5%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

!apt-get -y install cmake
!pip install llama-cpp-python langchain langchain-community huggingface_hub

In [ ]:
from unsloth import FastLanguageModel
from transformers import BitsAndBytesConfig
from transformers import TextStreamer
import torch
from datasets import load_dataset
# import llama_cpp
from llama_cpp import Llama
from huggingface_hub import login, hf_hub_download
import os

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


Vanila  LLaMA 3.1 모델 불러오기

In [ ]:
max_seq_length = 2048  # Choose any! We auto support RoPE Scaling internally!
dtype = None  # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+

# 8-bit quantization configuration
quantization_config = BitsAndBytesConfig(
    load_in_8bit=True,
    llm_int8_threshold=6.0,
    llm_int8_has_fp16_weight=False,
)

vanila_model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    quantization_config = quantization_config,
    )

==((====))==  Unsloth 2024.9.post4: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


재정 데이터를 활용해 Fine-tuning 된 모델 불러오기

In [ ]:
%%capture

login("")

# download fine_tuned model
model_path = hf_hub_download(
    repo_id="SoonKwan/model",
    filename="unsloth.Q8_0.gguf"
)

fine_tuned_model = Llama(
    model_path=model_path,
    n_ctx=2048,  # context window
    n_gpu_layers=-1  # auto detect GPU layers
)

In [ ]:
def generate_response(instruction, input_text="", output_text=""):

    prompt = alpaca_prompt.format(instruction, input_text, output_text)


    output = fine_tuned_model(
        prompt,
        max_tokens=128,
        temperature=0.7,
        echo=True
    )

    return output['choices'][0]['text'].strip()

Alpaca 프롬프트 템플릿 정의

In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}
### Input:
{}

### Response:
{}
"""

재정 데이터 질의응답 데이터셋 불러오기

In [ ]:
dataset = load_dataset("csv", data_files="train.csv", split = "train")

In [ ]:
instruction = dataset['Question'][0]

Answer Response

In [ ]:
dataset['Answer'][0]

'2024년 중앙정부 재정체계는 예산(일반·특별회계)과 기금으로 구분되며, 2024년 기준으로 일반회계 1개, 특별회계 21개, 기금 68개로 구성되어 있습니다.'

Vanila LLaMA 3.1 Response (1)

In [ ]:
%%capture

FastLanguageModel.for_inference(vanila_model)
vanila_inputs = tokenizer(
[
    alpaca_prompt.format(
        instruction, # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")


text_streamer = TextStreamer(tokenizer)
vanila_outputs = vanila_model.generate(
    **vanila_inputs,
    streamer = text_streamer,
    max_new_tokens = 128,
      )

vanila_response = tokenizer.decode(vanila_outputs[0], skip_special_tokens=True)

In [ ]:
print(vanila_response)

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
2024년 중앙정부 재정체계는 어떻게 구성되어 있나요?
### Input:


### Response:




Fine-tuning LLaMA 3.1 Response (1)

In [ ]:
fine_tuned_response = generate_response(instruction)

llama_perf_context_print:        load time =   34195.06 ms
llama_perf_context_print: prompt eval time =       0.00 ms /    53 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    49 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   97305.10 ms /   102 tokens


In [ ]:
print(fine_tuned_response)

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
2024년 중앙정부 재정체계는 어떻게 구성되어 있나요?
### Input:


### Response:

재정체계는 일반회계, 특별회계, 사립학교특별회계, 부처특별회계, 사립대학특별회계, 국립대학특별회계, 기타특별회계로 구성되어 있습니다.


Insturction 변형

In [ ]:
instruction = "2024년 중앙정부 재정체계 구성?"

Vanila LLaMA 3.1 Response (2)

In [ ]:
%%capture

FastLanguageModel.for_inference(vanila_model)
vanila_inputs = tokenizer(
[
    alpaca_prompt.format(
        instruction, # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")


text_streamer = TextStreamer(tokenizer)
vanila_outputs = vanila_model.generate(
    **vanila_inputs,
    streamer = text_streamer,
    max_new_tokens = 128,
      )

vanila_response = tokenizer.decode(vanila_outputs[0], skip_special_tokens=True)

In [ ]:
print(vanila_response)

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
2024년 중앙정부 재정체계 구성?
### Input:


### Response:

```
2024년 중앙정부 재정체계 구성은 다음과 같다.
1. 중앙정부 예산
2. 국회 예산
3. 지방정부 예산
4. 국회 예산
5. 국회 예산
6. 국회 예산
7. 국회 예산
8. 국회 예산
9. 국회 예산
10. 국회 예산
11. 국회 예산
12. 국회 예산
13. 국회 예산
14. 국회 예산
15. 국회 예산
16. 국회 예산



Fine-tuning LLaMA 3.1 Response (2)

In [ ]:
fine_tuned_response = generate_response(instruction)

Llama.generate: 40 prefix-match hit, remaining 8 prompt tokens to eval
llama_perf_context_print:        load time =   34195.06 ms
llama_perf_context_print: prompt eval time =       0.00 ms /     8 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    53 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   63882.01 ms /    61 tokens


In [ ]:
print(fine_tuned_response)

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
2024년 중앙정부 재정체계 구성?
### Input:


### Response:

중앙정부 재정체계는 일반회계, 일반회계 특별회계(세관고과금, 세입보전금, 국가재정운용비)와 특별회계(일반, 지방)로 구분됩니다.
